In [1]:
#############################################
# PROJE: Hybrid Recommender System
#############################################

# ID'si verilen kullanıcı için item-based ve user-based recomennder yöntemlerini kullanarak tahmin yapınız.
# 5 öneri user-based modelden 5 öneri de item-based modelden ele alınız ve nihai olarak 10 öneriyi 2 modelden yapınız.

#############################################
# Görev 1: Verinin Hazırlanması
#############################################

import pandas as pd
pd.pandas.set_option('display.max_columns', None)
pd.pandas.set_option('display.width', 100)

# Adım 1: Movie ve Rating veri setlerini okutunuz.
# movieId, film adı ve filmin tür bilgilerini içeren veri seti
movie = pd.read_csv('../input/movielens-20m-dataset/movie.csv')
movie.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [2]:
movie.shape

(27278, 3)

In [3]:
# UserID, film adı, filme verilen oy ve zaman bilgisini içeren veri seti
rating = pd.read_csv('../input/movielens-20m-dataset/rating.csv')
rating.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [4]:
rating.shape

(20000263, 4)

In [5]:
rating["userId"].nunique()

138493

In [6]:
# Adım 2: Rating veri setine filmlerin isimlerini ve türünü movie film setini kullanrak ekleyiniz.
# Ratingdeki kullanıcıların oy kullandıkları filmlerin sadece id'si var.
# Idlere ait film isimlerini ve türünü movie veri setinden ekliyoruz.
df = movie.merge(rating, how="left", on="movieId")
df.head(50)

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.0,4.0,1999-12-11 13:36:47
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6.0,5.0,1997-03-13 17:50:52
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8.0,4.0,1996-06-05 13:37:51
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10.0,4.0,1999-11-25 02:44:47
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11.0,4.5,2009-01-02 01:13:41
5,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,12.0,4.0,1997-03-22 20:48:38
6,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,13.0,4.0,1996-11-27 08:21:31
7,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,14.0,4.5,2008-10-29 20:13:59
8,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,16.0,3.0,2001-05-27 13:22:52
9,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,19.0,5.0,1997-02-05 21:03:48


In [7]:
df.shape

(20000797, 6)

In [8]:
# Adım 3: Herbir film için toplam kaç kişinin oy kullandığını hesaplayınız.Toplam oy kullanılma sayısı 1000'un altında olan filmleri veri setinden çıkarınız.
# Herbir film için toplam kaç kişinin oy kullanıldığını hesaplıyoruz.
comment_counts = pd.DataFrame(df["title"].value_counts())
comment_counts

,title
Pulp Fiction (1994),67310
Forrest Gump (1994),66172
"Shawshank Redemption, The (1994)",63366
"Silence of the Lambs, The (1991)",63299
Jurassic Park (1993),59715
...,...
Rapture (Arrebato) (1980),1
"Education of Mohammad Hussein, The (2013)",1
Satanas (2007),1
Psychosis (2010),1


In [9]:
# Toplam oy kullanılma sayısı 1000'in altında olan filmlerin isimlerini rare_movies de tutuyoruz.
# Ve veri setinden çıkartıyoruz
rare_movies = comment_counts[comment_counts["title"] <= 1000].index
common_movies = df[~df["title"].isin(rare_movies)]

In [10]:
common_movies.shape

(17766015, 6)

In [11]:
# Adım 4: # index'te userID'lerin sutunlarda film isimlerinin ve değer olarakta ratinglerin bulunduğu
# dataframe için pivot table oluşturunuz.

user_movie_df = common_movies.pivot_table(index=["userId"], columns=["title"], values="rating")
user_movie_df.head()

title   'burbs, The (1989)  (500) Days of Summer (2009)  *batteries not included (1987)  \
userId                                                                                    
1.0                    NaN                          NaN                             NaN   
2.0                    NaN                          NaN                             NaN   
3.0                    NaN                          NaN                             NaN   
4.0                    NaN                          NaN                             NaN   
5.0                    NaN                          NaN                             NaN   

title   ...And Justice for All (1979)  10 Things I Hate About You (1999)  10,000 BC (2008)  \
userId                                                                                       
1.0                               NaN                                NaN               NaN   
2.0                               NaN                                NaN               NaN   
3.0                               NaN                                NaN               NaN   
4.0                               NaN                                NaN               NaN   
5.0                               NaN                                NaN               NaN   

title   101 Dalmatians (1996)  101 Dalmatians (One Hundred and One Dalmatians) (1961)  \
userId                                                                                  
1.0                       NaN                                                NaN        
2.0                       NaN                                                NaN        
3.0                       NaN                                                NaN        
4.0                       NaN                                                NaN        
5.0                       NaN                                                NaN        

title   102 Dalmatians (2000)  12 Angry Men (1957)  12 Years a Slave (2013)  127 Hours (2010)  \
userId                                                                                          
1.0                       NaN                  NaN                      NaN               NaN   
2.0                       NaN                  NaN                      NaN               NaN   
3.0                       NaN                  NaN                      NaN               NaN   
4.0                       NaN                  NaN                      NaN               NaN   
5.0                       NaN                  NaN                      NaN               NaN   

title   13 Going on 30 (2004)  13th Warrior, The (1999)  1408 (2007)  15 Minutes (2001)  \
userId                                                                                    
1.0                       NaN                       NaN          NaN                NaN   
2.0                       NaN                       NaN          NaN                NaN   
3.0                       NaN                       NaN          NaN                NaN   
4.0                       NaN                       NaN          NaN                NaN   
5.0                       NaN                       NaN          NaN                NaN   

title   16 Blocks (2006)  17 Again (2009)  1984 (Nineteen Eighty-Four) (1984)  \
userId                                                                          
1.0                  NaN              NaN                                 NaN   
2.0                  NaN              NaN                                 NaN   
3.0                  NaN              NaN                                 5.0   
4.0                  NaN              NaN                                 NaN   
5.0                  NaN              NaN                                 NaN   

title   2 Days in the Valley (1996)  2 Fast 2 Furious (Fast and the Furious 2, The) (2003)  \
userId                                                                                       
1.0  

In [12]:
"""# Adım 5: Yukarıda yapılan tüm işlemleri fonksiyonlaştıralım
def create_user_movie_df():
    import pandas as pd
    movie = pd.read_csv('../input/movielens-20m-dataset/movie.csv')
    rating = pd.read_csv('../input/movielens-20m-dataset/rating.csv')
    df = movie.merge(rating, how="left", on="movieId")
    comment_counts = pd.DataFrame(df["title"].value_counts())
    rare_movies = comment_counts[comment_counts["title"] <= 1000].index
    common_movies = df[~df["title"].isin(rare_movies)]
    user_movie_df = common_movies.pivot_table(index=["userId"], columns=["title"], values="rating")
    return user_movie_df

user_movie_df = create_user_movie_df()"""

'# Adım 5: Yukarıda yapılan tüm işlemleri fonksiyonlaştıralım\ndef create_user_movie_df():\n    import pandas as pd\n    movie = pd.read_csv(\'../input/movielens-20m-dataset/movie.csv\')\n    rating = pd.read_csv(\'../input/movielens-20m-dataset/rating.csv\')\n    df = movie.merge(rating, how="left", on="movieId")\n    comment_counts = pd.DataFrame(df["title"].value_counts())\n    rare_movies = comment_counts[comment_counts["title"] <= 1000].index\n    common_movies = df[~df["title"].isin(rare_movies)]\n    user_movie_df = common_movies.pivot_table(index=["userId"], columns=["title"], values="rating")\n    return user_movie_df\n\nuser_movie_df = create_user_movie_df()'

In [13]:
#############################################
# Görev 2: Öneri Yapılacak Kullanıcının İzlediği Filmlerin Belirlenmesi
#############################################

# Adım 1: Rastgele bir kullanıcı id'si seçiniz.
random_user = 108170

In [14]:
# Adım 2: Seçilen kullanıcıya ait gözlem birimlerinden oluşan random_user_df adında yeni bir dataframe oluşturunuz.
random_user_df = user_movie_df[user_movie_df.index == random_user]
random_user_df.head()

title     'burbs, The (1989)  (500) Days of Summer (2009)  *batteries not included (1987)  \
userId                                                                                      
108170.0                 NaN                          NaN                             NaN   

title     ...And Justice for All (1979)  10 Things I Hate About You (1999)  10,000 BC (2008)  \
userId                                                                                         
108170.0                            NaN                                NaN               NaN   

title     101 Dalmatians (1996)  101 Dalmatians (One Hundred and One Dalmatians) (1961)  \
userId                                                                                    
108170.0                    NaN                                                NaN        

title     102 Dalmatians (2000)  12 Angry Men (1957)  12 Years a Slave (2013)  127 Hours (2010)  \
userId                                                                                            
108170.0                    NaN                  NaN                      NaN               NaN   

title     13 Going on 30 (2004)  13th Warrior, The (1999)  1408 (2007)  15 Minutes (2001)  \
userId                                                                                      
108170.0                    NaN                       NaN          NaN                NaN   

title     16 Blocks (2006)  17 Again (2009)  1984 (Nineteen Eighty-Four) (1984)  \
userId                                                                            
108170.0               NaN              NaN                                 NaN   

title     2 Days in the Valley (1996)  2 Fast 2 Furious (Fast and the Furious 2, The) (2003)  \
userId                                                                                         
108170.0                          NaN                                                NaN       

title     20,000 Leagues Under the Sea (1954)  200 Cigarettes (1999)  \
userId                                                                 
108170.0                                  NaN                    NaN   

title     2001: A Space Odyssey (1968)  2010: The Year We Make Contact (1984)  2012 (2009)  \
userId                                                                                       
108170.0                           5.0                                    NaN          NaN   

title     2046 (2004)  21 (2008)  21 Grams (2003)  21 Jump Street (2012)  \
userId                                                                     
108170.0          NaN        NaN              NaN                    NaN   

title     24 Hour Party People (2002)  25th Hour (2002)  27 Dresses (2008)  28 Days (2000)  \
userId                                                                                       
108170.0                          NaN               NaN                NaN             NaN   

title     28 Days Later (2002)  28 Weeks Later (2007)  3 Ninjas (1992)  3-Iron (Bin-jip) (2004)  \
userId                                                                                            
108170.0                   NaN                    NaN              NaN                      NaN   

title     30 Days of Night (2007)  300 (2007)  3000 Miles to Graceland (2001)  \
userId                                                                          
108170.0                      NaN         NaN                             NaN   

title     39 Steps, The (1935)  3:10 to Yuma (2007)  40 Days and 40 Nights (2002)  \
userId                                                                              
108170.0                   NaN                  NaN                           NaN   

title     40-Year-Old Virgin, The (2005)  400 Blows, The (Les quatre cents coups) (1959)  \
userId                                                                                     
108170.0                             NaN      

In [15]:
# Adım 3: Seçilen kullanıcının oy kullandığı filmleri movies_watched adında bir listeye atayınız.
movies_watched = random_user_df.columns[random_user_df.notna().any()].tolist()
movies_watched

['2001: A Space Odyssey (1968)',
 'Adventures of Priscilla, Queen of the Desert, The (1994)',
 'Akira (1988)',
 'Aladdin (1992)',
 'Aliens (1986)',
 'Almost Famous (2000)',
 'Along Came Polly (2004)',
 'Amadeus (1984)',
 'American History X (1998)',
 'Any Given Sunday (1999)',
 'Arachnophobia (1990)',
 'Army of Darkness (1993)',
 'Bad Santa (2003)',
 'Basic Instinct (1992)',
 'Batman Returns (1992)',
 'Big Lebowski, The (1998)',
 'Blair Witch Project, The (1999)',
 'Blues Brothers, The (1980)',
 'Boondock Saints, The (2000)',
 'Bourne Identity, The (2002)',
 'Braveheart (1995)',
 'Breakfast Club, The (1985)',
 "Carlito's Way (1993)",
 'Catwoman (2004)',
 "Charlie's Angels (2000)",
 'City Slickers (1991)',
 "City Slickers II: The Legend of Curly's Gold (1994)",
 'Clear and Present Danger (1994)',
 'Clerks (1994)',
 'Clockwork Orange, A (1971)',
 'Con Air (1997)',
 'Contact (1997)',
 'Copycat (1995)',
 'Crocodile Dundee (1986)',
 'Cruel Intentions (1999)',
 'Crying Game, The (1992)',
 'D

In [16]:
#############################################
# Görev 3: Aynı Filmleri İzleyen Diğer Kullanıcıların Verisine ve Id'lerine Erişmek
#############################################

# Adım 1: Seçilen kullanıcının izlediği fimlere ait sutunları user_movie_df'ten seçiniz ve movies_watched_df adında yeni bir dataframe oluşturuyoruz.
movies_watched_df = user_movie_df[movies_watched]
movies_watched_df.head()

title,2001: A Space Odyssey (1968),"Adventures of Priscilla, Queen of the Desert, The (1994)",Akira (1988),Aladdin (1992),Aliens (1986),Almost Famous (2000),Along Came Polly (2004),Amadeus (1984),American History X (1998),Any Given Sunday (1999),Arachnophobia (1990),Army of Darkness (1993),Bad Santa (2003),Basic Instinct (1992),Batman Returns (1992),"Big Lebowski, The (1998)","Blair Witch Project, The (1999)","Blues Brothers, The (1980)","Boondock Saints, The (2000)","Bourne Identity, The (2002)",Braveheart (1995),"Breakfast Club, The (1985)",Carlito's Way (1993),Catwoman (2004),Charlie's Angels (2000),City Slickers (1991),City Slickers II: The Legend of Curly's Gold (1994),Clear and Present Danger (1994),Clerks (1994),"Clockwork Orange, A (1971)",Con Air (1997),Contact (1997),Copycat (1995),Crocodile Dundee (1986),Cruel Intentions (1999),"Crying Game, The (1992)","Dark Crystal, The (1982)",Dawn of the Dead (2004),Dead Poets Society (1989),Deliverance (1972),Demolition Man (1993),Desperado (1995),"Devil's Advocate, The (1997)",Dirty Rotten Scoundrels (1988),Donnie Darko (2001),Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964),Easy Rider (1969),Edward Scissorhands (1990),Eternal Sunshine of the Spotless Mind (2004),Evil Dead II (Dead by Dawn) (1987),Ferris Bueller's Day Off (1986),Fight Club (1999),Finding Nemo (2003),Finding Neverland (2004),First Knight (1995),"Fisher King, The (1991)",Flight of the Navigator (1986),Freddy Got Fingered (2001),"Frighteners, The (1996)",From Dusk Till Dawn (1996),Gangs of New York (2002),Gattaca (1997),Ghostbusters (a.k.a. Ghost Busters) (1984),Gladiator (2000),Goldfinger (1964),Goodfellas (1990),"Grand Day Out with Wallace and Gromit, A (1989)",Grease (1978),Grease 2 (1982),"Green Mile, The (1999)",Grosse Pointe Blank (1997),Harry Potter and the Prisoner of Azkaban (2004),Hero (Ying xiong) (2002),Highlander (1986),"Honey, I Shrunk the Kids (1989)","Hunt for Red October, The (1990)",Indiana Jones and the Last Crusade (1989),"Italian Job, The (2003)",Kill Bill: Vol. 1 (2003),Kill Bill: Vol. 2 (2004),Kindergarten Cop (1990),Kingpin (1996),"Knight's Tale, A (2001)",Labyrinth (1986),Last Action Hero (1993),"Last Samurai, The (2003)",Leaving Las Vegas (1995),Lemony Snicket's A Series of Unfortunate Events (2004),Liar Liar (1997),Life Is Beautiful (La Vita è bella) (1997),"Little Mermaid, The (1989)",Live and Let Die (1973),"Lord of the Rings: The Fellowship of the Ring, The (2001)","Lord of the Rings: The Return of the King, The (2003)","Lord of the Rings: The Two Towers, The (2002)",Léon: The Professional (a.k.a. The Professional) (Léon) (1994),Master and Commander: The Far Side of the World (2003),"Matrix, The (1999)",Meet the Parents (2000),Memento (2000),Mickey Blue Eyes (1999),Minority Report (2002),Misery (1990),Mission: Impossible (1996),Mission: Impossible II (2000),"Monsters, Inc. (2001)",Monty Python Live at the Hollywood Bowl (1982),Monty Python and the Holy Grail (1975),Monty Python's Life of Brian (1979),Mulholland Drive (2001),Muriel's Wedding (1994),My Cousin Vinny (1992),"Naked Gun: From the Files of Police Squad!, The (1988)",Nine Months (1995),Ocean's Eleven (2001),"Others, The (2001)",Pirates of the Caribbean: The Curse of the Black Pearl (2003),Police Academy 5: Assignment: Miami Beach (1988),Police Academy 6: City Under Siege (1989),"Princess Bride, The (1987)",Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981),Rain Man (1988),Reservoir Dogs (1992),Ringu (Ring) (1998),Rob Roy (1995),Robin Hood: Men in Tights (1993),Romancing the Stone (1984),Rumble in the Bronx (Hont faan kui) (1995),Saving Private Ryan (1998),Scarface (1983),Schindler's List (1993),"Secret of NIMH, The (1982)",Seven (a.k.a. Se7en) (1995),Shaun of the Dead (2004),"Shawshank Redemption, The (1994)","Shining, The (1980)",Shrek (2001),Shrek 2 (2004),"Silence of the Lambs, The (1991)","Sixth Sense, The (1999)",Snatch (2000),Sneakers (1992),So I Married an Axe Mur

In [17]:
movies_watched_df.shape

(138493, 186)

In [18]:
# Adım 2: Herbir kullancının seçili user'in izlediği filmlerin kaçını izlediği bilgisini taşıyan user_movie_count adında yeni bir dataframe oluşturunuz.
# Ve yeni bir df oluşturuyoruz.
user_movie_count = movies_watched_df.T.notnull().sum()
user_movie_count = user_movie_count.reset_index()
user_movie_count.columns = ["userId", "movie_count"]
user_movie_count.head(50)

,userId,movie_count
0,1.0,53
1,2.0,11
2,3.0,47
3,4.0,5
4,5.0,16
5,6.0,4
6,7.0,31
7,8.0,13
8,9.0,4
9,10.0,13


In [19]:
# Adım 3: Seçilen kullanıcının oy verdiği filmlerin yüzde 60 ve üstünü izleyenleri benzer kullanıcılar olarak görüyoruz.
# Bu kullanıcıların id’lerinden users_same_movies adında bir liste oluşturunuz.
perc = len(movies_watched) * 60 / 100
users_same_movies = user_movie_count[user_movie_count["movie_count"] > perc]["userId"]
len(users_same_movies)

2326

In [20]:
#############################################
# Görev 4: Öneri Yapılacak Kullanıcı ile En Benzer Kullanıcıların Belirlenmesi
#############################################

# Adım 1: user_same_movies listesi içerisindeki seçili user ile benzerlik gösteren kullanıcıların id’lerinin bulunacağı şekilde movies_watched_df dataframe’ini filtreleyiniz.
final_df = movies_watched_df[movies_watched_df.index.isin(users_same_movies)]
final_df.head()

title,2001: A Space Odyssey (1968),"Adventures of Priscilla, Queen of the Desert, The (1994)",Akira (1988),Aladdin (1992),Aliens (1986),Almost Famous (2000),Along Came Polly (2004),Amadeus (1984),American History X (1998),Any Given Sunday (1999),Arachnophobia (1990),Army of Darkness (1993),Bad Santa (2003),Basic Instinct (1992),Batman Returns (1992),"Big Lebowski, The (1998)","Blair Witch Project, The (1999)","Blues Brothers, The (1980)","Boondock Saints, The (2000)","Bourne Identity, The (2002)",Braveheart (1995),"Breakfast Club, The (1985)",Carlito's Way (1993),Catwoman (2004),Charlie's Angels (2000),City Slickers (1991),City Slickers II: The Legend of Curly's Gold (1994),Clear and Present Danger (1994),Clerks (1994),"Clockwork Orange, A (1971)",Con Air (1997),Contact (1997),Copycat (1995),Crocodile Dundee (1986),Cruel Intentions (1999),"Crying Game, The (1992)","Dark Crystal, The (1982)",Dawn of the Dead (2004),Dead Poets Society (1989),Deliverance (1972),Demolition Man (1993),Desperado (1995),"Devil's Advocate, The (1997)",Dirty Rotten Scoundrels (1988),Donnie Darko (2001),Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964),Easy Rider (1969),Edward Scissorhands (1990),Eternal Sunshine of the Spotless Mind (2004),Evil Dead II (Dead by Dawn) (1987),Ferris Bueller's Day Off (1986),Fight Club (1999),Finding Nemo (2003),Finding Neverland (2004),First Knight (1995),"Fisher King, The (1991)",Flight of the Navigator (1986),Freddy Got Fingered (2001),"Frighteners, The (1996)",From Dusk Till Dawn (1996),Gangs of New York (2002),Gattaca (1997),Ghostbusters (a.k.a. Ghost Busters) (1984),Gladiator (2000),Goldfinger (1964),Goodfellas (1990),"Grand Day Out with Wallace and Gromit, A (1989)",Grease (1978),Grease 2 (1982),"Green Mile, The (1999)",Grosse Pointe Blank (1997),Harry Potter and the Prisoner of Azkaban (2004),Hero (Ying xiong) (2002),Highlander (1986),"Honey, I Shrunk the Kids (1989)","Hunt for Red October, The (1990)",Indiana Jones and the Last Crusade (1989),"Italian Job, The (2003)",Kill Bill: Vol. 1 (2003),Kill Bill: Vol. 2 (2004),Kindergarten Cop (1990),Kingpin (1996),"Knight's Tale, A (2001)",Labyrinth (1986),Last Action Hero (1993),"Last Samurai, The (2003)",Leaving Las Vegas (1995),Lemony Snicket's A Series of Unfortunate Events (2004),Liar Liar (1997),Life Is Beautiful (La Vita è bella) (1997),"Little Mermaid, The (1989)",Live and Let Die (1973),"Lord of the Rings: The Fellowship of the Ring, The (2001)","Lord of the Rings: The Return of the King, The (2003)","Lord of the Rings: The Two Towers, The (2002)",Léon: The Professional (a.k.a. The Professional) (Léon) (1994),Master and Commander: The Far Side of the World (2003),"Matrix, The (1999)",Meet the Parents (2000),Memento (2000),Mickey Blue Eyes (1999),Minority Report (2002),Misery (1990),Mission: Impossible (1996),Mission: Impossible II (2000),"Monsters, Inc. (2001)",Monty Python Live at the Hollywood Bowl (1982),Monty Python and the Holy Grail (1975),Monty Python's Life of Brian (1979),Mulholland Drive (2001),Muriel's Wedding (1994),My Cousin Vinny (1992),"Naked Gun: From the Files of Police Squad!, The (1988)",Nine Months (1995),Ocean's Eleven (2001),"Others, The (2001)",Pirates of the Caribbean: The Curse of the Black Pearl (2003),Police Academy 5: Assignment: Miami Beach (1988),Police Academy 6: City Under Siege (1989),"Princess Bride, The (1987)",Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981),Rain Man (1988),Reservoir Dogs (1992),Ringu (Ring) (1998),Rob Roy (1995),Robin Hood: Men in Tights (1993),Romancing the Stone (1984),Rumble in the Bronx (Hont faan kui) (1995),Saving Private Ryan (1998),Scarface (1983),Schindler's List (1993),"Secret of NIMH, The (1982)",Seven (a.k.a. Se7en) (1995),Shaun of the Dead (2004),"Shawshank Redemption, The (1994)","Shining, The (1980)",Shrek (2001),Shrek 2 (2004),"Silence of the Lambs, The (1991)","Sixth Sense, The (1999)",Snatch (2000),Sneakers (1992),So I Married an Axe Mur

In [21]:
final_df.shape

(2326, 186)

In [22]:
# Adım 2: Kullanıcıların birbirleri ile olan korelasyonlarının bulunacağı yeni bir corr_df dataframe’i oluşturunuz.
corr_df = final_df.T.corr().unstack().sort_values()
corr_df = pd.DataFrame(corr_df, columns=["corr"])
corr_df.index.names = ['user_id_1', 'user_id_2']
corr_df = corr_df.reset_index()
corr_df[corr_df["user_id_1"] == random_user]

,user_id_1,user_id_2,corr
23452,108170.0,89242.0,-0.138100
31228,108170.0,25614.0,-0.122044
40470,108170.0,12490.0,-0.106929
50270,108170.0,87954.0,-0.094216
52070,108170.0,122980.0,-0.092017
...,...,...,...
5398945,108170.0,121747.0,0.673295
5404112,108170.0,119285.0,0.700081
5405004,108170.0,11517.0,0.707652
5405810,108170.0,5155.0,0.716406


In [23]:
# Adım 3: Seçili kullanıcı ile yüksek korelasyona sahip (0.65’in üzerinde olan) kullanıcıları filtreleyerek top_users adında yeni bir dataframe oluşturunuz.
top_users = corr_df[(corr_df["user_id_1"] == random_user) & (corr_df["corr"] >= 0.65)][["user_id_2", "corr"]].reset_index(drop=True)
top_users = top_users.sort_values(by='corr', ascending=False)
top_users.rename(columns={"user_id_2": "userId"}, inplace=True)

In [24]:
top_users.shape

(7, 2)

In [25]:

# Adım 4:  top_users dataframe’ine rating veri seti ile merge ediniz
top_users_ratings = top_users.merge(rating[["userId", "movieId", "rating"]], how='inner')
top_users_ratings = top_users_ratings[top_users_ratings["userId"] != random_user]
top_users_ratings["userId"].unique()
top_users_ratings.head()

,userId,corr,movieId,rating
194,5155.0,0.716406,1,3.5
195,5155.0,0.716406,2,3.0
196,5155.0,0.716406,5,3.0
197,5155.0,0.716406,9,3.0
198,5155.0,0.716406,10,4.0


In [26]:
#############################################
# Görev 5: Weighted Average Recommendation Score'un Hesaplanması ve İlk 5 Filmin Tutulması
#############################################

# Adım 1: Her bir kullanıcının corr ve rating değerlerinin çarpımından oluşan weighted_rating adında yeni bir değişken oluşturunuz.
top_users_ratings['weighted_rating'] = top_users_ratings['corr'] * top_users_ratings['rating']

In [27]:
# Adım 2: Film id’sive her bir filme ait tüm kullanıcıların weighted rating’lerinin ortalama değerini içeren recommendation_df adında yeni bir
# dataframe oluşturunuz.
recommendation_df = top_users_ratings.groupby('movieId').agg({"weighted_rating": "mean"})
recommendation_df = recommendation_df.reset_index()
recommendation_df.head()

,movieId,weighted_rating
0,1,2.124196
1,2,1.319527
2,3,0.986312
3,5,1.484503
4,6,2.526375


In [28]:
# Adım 3: Adım3: recommendation_df içerisinde weighted rating'i 3.5'ten büyük olan filmleri seçiniz ve weighted rating’e göre sıralayınız.
# İlk 5 gözlemi movies_to_be_recommend olarak kaydediniz.
recommendation_df[recommendation_df["weighted_rating"] > 3.5]
movies_to_be_recommend = recommendation_df[recommendation_df["weighted_rating"] > 3.5].sort_values("weighted_rating", ascending=False)

In [29]:
# Adım 4:  Tavsiye edilen 5 filmin isimlerini getiriniz.
movies_to_be_recommend.merge(movie[["movieId", "title"]])["title"]

# 0    Mystery Science Theater 3000: The Movie (1996)
# 1                               Natural, The (1984)
# 2                             Super Troopers (2001)
# 3                         Christmas Story, A (1983)
# 4                       Sonatine (Sonachine) (1993)

0        Mystery Science Theater 3000: The Movie (1996)
1                                   Natural, The (1984)
2                                 Super Troopers (2001)
3                             Christmas Story, A (1983)
4                           Sonatine (Sonachine) (1993)
5                           The Raid: Redemption (2011)
6                                       Watchmen (2009)
7                                 28 Weeks Later (2007)
8     Eight Deadly Shots (Kahdeksan surmanluotia) (1...
9       Violent Cop (Sono otoko, kyôbô ni tsuki) (1989)
10     Unknown Soldier, The (Tuntematon sotilas) (1955)
11    Porco Rosso (Crimson Pig) (Kurenai no buta) (1...
12    Spring, Summer, Fall, Winter... and Spring (Bo...
13                              Band of Brothers (2001)
14     Once Upon a Time in China (Wong Fei Hung) (1991)
15                                      Kid, The (1921)
16                              12 Years a Slave (2013)
17                   Show Me Love (Fucking Åmål)

In [30]:
#############################################
# Adım 6: Item-Based Recommendation
#############################################

# Kullanıcının en son izlediği ve en yüksek puan verdiği filmin adına göre item-based öneri yapınız.
user = 108170

# Adım 1: movie,rating veri setlerini okutunuz.
movie = pd.read_csv('../input/movielens-20m-dataset/movie.csv')
rating = pd.read_csv('../input/movielens-20m-dataset/rating.csv')

In [31]:

# Adım 2: Öneri yapılacak kullanıcının 5 puan verdiği filmlerden puanı en güncel olan filmin id'sinin alınız.
movie_id = rating[(rating["userId"] == user) & (rating["rating"] == 5.0)].sort_values(by="timestamp", ascending=False)["movieId"][0:1].values[0]

In [32]:

# Adım 3 :User based recommendation bölümünde oluşturulan user_movie_df dataframe’ini seçilen film id’sine göre filtreleyiniz.
movie_df = user_movie_df[movie[movie["movieId"] == movie_id]["title"].values[0]]

In [33]:

# Adım 4: Filtrelenen dataframe’i kullanarak seçili filmle diğer filmlerin korelasyonunu bulunuz ve sıralayınız.
user_movie_df.corrwith(movie_df).sort_values(ascending=False).head(10)

title
Wild at Heart (1990)                     1.000000
My Science Project (1985)                0.570187
Mediterraneo (1991)                      0.538868
Old Man and the Sea, The (1958)          0.536192
National Lampoon's Senior Trip (1995)    0.533029
Clockwatchers (1997)                     0.483337
Repo Man (1984)                          0.478674
Lost Highway (1997)                      0.476251
Blue Velvet (1986)                       0.471225
Jeffrey (1995)                           0.457849
dtype: float64

In [34]:

# Son iki adımı uygulayan fonksiyon
def item_based_recommender(movie_name, user_movie_df):
    movie = user_movie_df[movie_name]
    return user_movie_df.corrwith(movie).sort_values(ascending=False).head(10)

In [35]:

# Adım 5: Seçili film’in kendisi haricinde ilk 5 film’I öneri olarak veriniz.
movies_from_item_based = item_based_recommender(movie[movie["movieId"] == movie_id]["title"].values[0], user_movie_df)

In [36]:
# 1'den 6'ya kadar. 0'da filmin kendisi var. Onu dışarda bıraktık.
movies_from_item_based[1:6].index


Index(['My Science Project (1985)', 'Mediterraneo (1991)', 'Old Man and the Sea, The (1958)',
       'National Lampoon's Senior Trip (1995)', 'Clockwatchers (1997)'],
      dtype='object', name='title')